In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib
import numpy as np

from rl_trading.simulation.env import StockExchangeEnv1

Using matplotlib backend: <object object at 0x7ff08150a8d0>


In [3]:
from rl_trading.data.indicators import *

state_config = {
        'market_state': ['vwap'],
        'technical_indicators': [
            (EMA, dict(timeperiod=5, normalize=True), '1min'),
            (EMA, dict(timeperiod=13, normalize=True), '1min'),
            (RSI, dict(timeperiod=7, normalize=True), '1min'),
            (BBANDS, dict(timeperiod=10), '1min'),
            (EMA, dict(timeperiod=20, normalize=True), '1h'),
            (EMA, dict(timeperiod=50, normalize=True), '1h'),
            (RSI, dict(timeperiod=14, normalize=True), '1h'),
            (BBANDS, dict(timeperiod=20), '1h'),
            (MACD_DIFF, dict(fastperiod=12, slowperiod=26, signalperiod=9, normalize=True), '1h'),
            (EMA, dict(timeperiod=50, normalize=True), '1d'),
            (EMA, dict(timeperiod=200, normalize=True), '1d'),
            (RSI, dict(timeperiod=14, normalize=True), '1d'),
            (BBANDS, dict(timeperiod=20), '1d'),
            (MACD_DIFF, dict(fastperiod=12, slowperiod=26, signalperiod=9, normalize=True), '1d'),
        ]}

In [ ]:
from ray.rllib.algorithms.sac import SACConfig

sac = (
    SACConfig()
    .training(
        initial_alpha=0.1,
        training_intensity=1,
        q_model_config={'fcnet_activation': 'tanh'},
        policy_model_config={'fcnet_activation': 'tanh'}
    )
    .environment(env='StockExchangeEnv-v1')
    .rollouts(num_rollout_workers=1, num_envs_per_worker=1)
    .resources(num_gpus=1)
    .reporting(min_sample_timesteps_per_iteration=1440)
    .evaluation(evaluation_interval=5, evaluation_duration=20, evaluation_config={'explore': False})
    .build()
)

for i in range(10):
    sac.train()
    eval_result = sac.evaluate()
    print(f'Episode: {i}, return: ', eval_result['evaluation']['episode_reward_mean'])

In [4]:
from ray.rllib.algorithms.algorithm import Algorithm

sac = Algorithm.from_checkpoint('/home/fassty/Devel/school/diploma_thesis/code/exp_results/continuous_env/SAC/SAC_1M_iterations_fee1e-3/SAC_StockExchangeEnv-v1_e6ac2_00000_0_2023-05-27_23-24-53/checkpoint_000050')

(RolloutWorker pid=2171749) 2023-05-28 18:08:30,912	WARNING env.py:155 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=2169299) 2023-05-28 18:07:17,183	ERROR worker.py:844 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=2169299, ip=192.168.0.222, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f3826b5a6e0>) [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(RolloutWorker pid=2169299)   File "/home/fassty/anaconda3/envs/diploma_thesis/lib/python3.10/site-packages/gymnasium/envs/registration.py", line 569, in make [repeated 3x across cluster]
(RolloutWorker pid=2169299)     _check_version_exists(ns,

In [8]:
env = StockExchangeEnv1(state_config=state_config,
                        exchange_config={'maker_fee': 1e-3})
state = env.reset(seed=52)[0]

for i in range(1000):
    env.render()
    action = sac.compute_single_action(state, explore=True)
    state, reward, done, _, _ = env.step(action)

KeyboardInterrupt: 

In [12]:
sac.get_policy().model

SACTorchModel(
  (action_model): FullyConnectedNetwork(
    (_logits): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=256, out_features=2, bias=True)
      )
    )
    (_hidden_layers): Sequential(
      (0): SlimFC(
        (_model): Sequential(
          (0): Linear(in_features=17, out_features=256, bias=True)
          (1): Tanh()
        )
      )
      (1): SlimFC(
        (_model): Sequential(
          (0): Linear(in_features=256, out_features=256, bias=True)
          (1): Tanh()
        )
      )
    )
    (_value_branch): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=256, out_features=1, bias=True)
      )
    )
  )
  (q_net): FullyConnectedNetwork(
    (_logits): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=256, out_features=1, bias=True)
      )
    )
    (_hidden_layers): Sequential(
      (0): SlimFC(
        (_model): Sequential(
          (0): Linear(in_features=18, out_features=256, bias=True)
          